# Best Locations for Fast Food in Downtown Chicago



## Intro
    Suppose one were to open a new fast food restaurant in the downtown area of Chicago,IL. The main intent of this restaurant would be speed and ease of access. The main target for this bussiness is commuter traffic and the local lunch crowd.
    
    The target for this project would be either chains or small business owners that are aiming to open a new location in downtown Chicago that fit the criteria (fast and high volume). 
    


## Code 

In [12]:
import requests
import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans

In [185]:
CLIENT_ID = 'B4U2MZYSR1BHH4SH5B5K1NEQRYKBRFRVXFBBTEIUAM2WPAK3' # your Foursquare ID
CLIENT_SECRET = 'ZNRMEUKU45XWHJDYQBPA2WQVCPVPYFVONLAPPY0VV12XJB4E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 10000

In [186]:
address = '500 West Madison St, Chicago, IL'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

radius = 1500

search_query = 'CTA'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
search_query = 'Fast food'
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
CTAresults = requests.get(url).json()
FFresults = requests.get(url2).json()

In [187]:
CTAven = CTAresults['response']['venues']
FFven = FFresults['response']['venues']

CTAdf = pd.json_normalize(CTAven)
FFdf = pd.json_normalize(FFven)

CTAbus = CTAdf[CTAdf['name'].str.contains("Bus")]
CTAtrain = CTAdf[~CTAdf['name'].str.contains("Bus|bus")]

In [188]:
def filter_dataframe(df):

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = df.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
            try:
                categories_list = row['categories']
            except:
                categories_list = row['venue.categories']
        
            if len(categories_list) == 0:
                return None
            else:
                return categories_list[0]['name']

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    return dataframe_filtered

In [189]:
Busdf1 = filter_dataframe(CTAbus)
Traindf1 = filter_dataframe(CTAtrain)
FastFooddf1 = filter_dataframe(FFdf)

In [190]:
address = '50 East Madison St, Chicago, IL'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

radius = 1500

search_query = 'CTA'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
search_query = 'Fast food'
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
CTAresults = requests.get(url).json()
FFresults = requests.get(url2).json()

In [191]:
CTAven = CTAresults['response']['venues']
FFven = FFresults['response']['venues']

CTAdf = pd.json_normalize(CTAven)
FFdf = pd.json_normalize(FFven)

CTAbus = CTAdf[CTAdf['name'].str.contains("Bus")]
CTAtrain = CTAdf[~CTAdf['name'].str.contains("Bus|bus")]

In [192]:
Busdf2 = filter_dataframe(CTAbus)
Traindf2 = filter_dataframe(CTAtrain)
FastFooddf2 = filter_dataframe(FFdf)

In [209]:
FastFooddf

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id,neighborhood
0,Firenze - Italian Street Food,Italian Restaurant,131 N Clinton St Ste 20,41.884020,-87.640540,"[{'label': 'display', 'lat': 41.88402, 'lng': ...",27,60661,US,Chicago,IL,United States,"[131 N Clinton St Ste 20, Chicago, IL 60661, U...",NaN,5a9ef8730a464d105d8f0773,NaN
1,Fast Track,Diner,629 W Lake St,41.885470,-87.644071,"[{'label': 'display', 'lat': 41.88546951693589...",350,60661,US,Chicago,IL,United States,"[629 W Lake St (at Desplaines St.), Chicago, I...",at Desplaines St.,4b8c6861f964a52045cf32e3,NaN
2,Fast Frame - River North,Frame Store,300 W Grand Ave #3,41.891773,-87.636010,"[{'label': 'display', 'lat': 41.89177257127777...",965,60610,US,Chicago,IL,United States,"[300 W Grand Ave #3 (at N Franklin St), Chicag...",at N Franklin St,4bc76b7215a7ef3b915e79da,NaN
3,FASTSIGNS,Design Studio,180 N Wacker Dr,41.885165,-87.636970,"[{'label': 'display', 'lat': 41.88516514544391...",331,60606,US,Chicago,IL,United States,"[180 N Wacker Dr, Chicago, IL 60606, United St...",NaN,4fe486d74c7fada8d19629f0,NaN
4,Chicago Union Station Food Court,Food Court,225 S Canal St,41.879339,-87.638738,"[{'label': 'display', 'lat': 41.879339217638, ...",514,60606,US,Chicago,IL,United States,"[225 S Canal St (at Adams St.), Chicago, IL 60...",at Adams St.,4c489f3a19fde21e8b460c76,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,Pierogi Street Food Truck + Eatery,Polish Restaurant,NaN,41.880898,-87.630806,"[{'label': 'display', 'lat': 41.88089810608153...",430,NaN,US,Chicago,IL,United States,"[Chicago, IL, United States]",NaN,55df5d1a498e5edc850462b5,NaN
46,University Food Services,Food,315 S Plymouth Ct,41.877998,-87.628609,"[{'label': 'display', 'lat': 41.87799835205078...",515,60604,US,Chicago,IL,United States,"[315 S Plymouth Ct, Chicago, IL 60604, United ...",NaN,4f3274d519836c91c7d99a74,NaN
47,Southern Pitch Food Truck,Food Truck,NaN,41.877972,-87.625678,"[{'label': 'display', 'lat': 41.87797181687470...",467,60608,US,Chicago,IL,United States,"[Chicago, IL 60608, United States]",NaN,543d6ed8498e707a14332557,NaN
48,Revival Food Hall,Food Court,125 S Clark St,41.879560,-87.630550,"[{'label': 'display', 'lat': 41.87956, 'lng': ...",482,60603,US,Chicago,IL,United States,"[125 S Clark St (W Adams St), Chicago, IL 6060...",W Adams St,5751af74cd102f915c2cb3de,NaN


In [204]:
Busdf = pd.concat([Busdf1, Busdf2])
Traindf = pd.concat([Traindf1, Traindf2])
FastFooddf = pd.concat([FastFooddf1, FastFooddf2])

In [211]:
myMap = folium.Map(location = [Traindf['lat'].iloc[0],Traindf['lng'].iloc[0]], zoom_start= 15)
myFoodMap = folium.Map(location = [Traindf['lat'].iloc[0],Traindf['lng'].iloc[0]], zoom_start= 15)
for i in range(0, len(Traindf)):
    folium.CircleMarker([Traindf['lat'].iloc[i],Traindf['lng'].iloc[i]], radius = 5, color = 'blue', fill = True, fill_color='blue', popup=Traindf['name'].iloc[i]).add_to(myMap)

for i in range(0, len(Busdf)):
    folium.CircleMarker([Busdf['lat'].iloc[i],Busdf['lng'].iloc[i]], radius = 3, color = 'red', fill = True, fill_color='red', popup=Busdf['name'].iloc[i]).add_to(myMap)

for i in range(0, len(FastFooddf)):
    folium.CircleMarker([FastFooddf['lat'].iloc[i],FastFooddf['lng'].iloc[i]], radius = 4, color = 'green', fill = True, fill_color='green').add_to(myMap)

myMap

In [174]:
def getKMcluster(df):
    kmeans = KMeans(init="k-means++", n_clusters=1, n_init=12)
    X = np.vstack((df['lat'],df['lng'])).T
    kmeans.fit(X)
    kmClusterCenters = kmeans.cluster_centers_
    return kmClusterCenters

In [212]:
X1 = getKMcluster(Traindf)
X2 = getKMcluster(Busdf)
X3 = getKMcluster(FastFooddf)

In [213]:
Thismap = folium.Map(location = [X1[0][0],X1[0][1]], zoom_start= 15)
folium.CircleMarker([X1[0][0],X1[0][1]], radius = 7, color = 'red', fill = True, fill_color='red', popup="Trains").add_to(Thismap)
folium.CircleMarker([X2[0][0],X2[0][1]], radius = 7, color = 'blue', fill = True, fill_color='red', popup="Busses").add_to(Thismap)
folium.CircleMarker([X3[0][0],X3[0][1]], radius = 7, color = 'green', fill = True, fill_color='red', popup="Fast Food").add_to(Thismap)

Thismap